In [66]:
bucketName = 'noisy-images'
imageName = 'blur-argo-logo.png'

In [68]:
from skimage import measure
import matplotlib.pyplot as plt
import numpy as np
import cv2
import asyncio
from nats.aio.client import Client as NATS
from nats.aio.errors import ErrConnectionClosed, ErrTimeout, ErrNoServers 

from minio import Minio
from minio.error import ResponseError

bucketName = 'noisy-images'

minioClient = Minio('minio-service.argo-events.svc:9000',
                  access_key='minio',
                  secret_key='minio123',
                  secure=False)

In [69]:
def mse(imageA, imageB):
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])
    return err

def compare_images(imageA, imageB, title):
    s =  measure.compare_ssim(imageA, imageB)
    return s

In [70]:
# Fetch from minio the noisy
# Get a full object and prints the original object stat information.
try:
    print(minioClient.fget_object(bucketName, imageName, 'out1/blur-argo-logo.png'))
except ResponseError as err:
    print(err)

<Object: bucket_name: noisy-images object_name: b'blur-argo-logo.png' last_modified: time.struct_time(tm_year=2020, tm_mon=3, tm_mday=18, tm_hour=4, tm_min=26, tm_sec=1, tm_wday=2, tm_yday=78, tm_isdst=0) etag: ec17069811594cea0a111bf27301e282-1 size: 104342 content_type: application/octet-stream, is_dir: False, metadata: {'Content-Type': 'application/octet-stream'}>


In [71]:
original = cv2.imread("out1/argo-logo.png")
noisy = cv2.imread("out1/blur-argo-logo.png")

In [72]:
original = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
noisy = cv2.cvtColor(noisy, cv2.COLOR_BGR2GRAY)

In [73]:
output = compare_images(original, noisy, "Original vs. Noisy")
print('Structural Similarity Measure is: ', output)

Structural Similarity Measure is:  0.8517659500290427


/home/vaibhav/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
  import sys


In [74]:
nc = NATS()

loop = asyncio.get_event_loop()

await nc.connect("nats.argo-events.svc:4222", loop=loop)

msg = ''

if output > 0.8:
    msg = 'SUCCESS: ' + str(output)
else:
    msg = 'FAILURE: ' + str(output)

await nc.publish("image-match", msg.encode())
